# 0.导包、导数据

In [7]:
import xgboost as xgb
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import csc_matrix

from sklearn.ensemble import RandomForestClassifier  
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import log_loss

pd.set_option('display.max_colwidth',1000)

In [2]:
# 取最后一天的作为验证
data = pd.read_pickle('data/round1_train2')
feature_train = data[data['f16'] <= 1537718400]
feature_train.drop('label',axis = 1,inplace = True)
feature_val = data[data['f16'] > 1537718400]
feature_val.drop('label',axis = 1,inplace = True)

label_train = data[data['f16'] <= 1537718400]['label']
label_val = data[data['f16'] > 1537718400]['label']

data = pd.read_pickle('data/round1_test2')
feature_test = data

data_index = pd.DataFrame(np.genfromtxt('data/round1_test.txt',dtype = np.str, delimiter=' ',skip_header=1))[0]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## 1.训练SVM

In [45]:
from sklearn import svm,datasets
from sklearn.metrics import log_loss

#调用SVC()
# 参数见：http://blog.csdn.net/xiaodongxiexie/article/details/70667101
clf = svm.SVC(
            C=0.001, 
            kernel='rbf', 
            degree=3, 
            gamma='auto', 
            coef0=0.0, 
            probability=False, 
            shrinking=True, 
            tol=0.001, 
            cache_size=200, 
            class_weight='balanced', 
            verbose=True, 
            max_iter=1000, 
            decision_function_shape='ovr', 
            random_state=None
)
#载入鸢尾花数据集
iris = datasets.load_iris()

#fit()训练
clf.fit(feature_train,label_train)

#predict()预测
label_val_svm = clf.predict(feature_val)
print(log_loss(label_val,label_val_svm))

[LibSVM]

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


33.957631066


## 2.训练逻辑回归

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss


feature_train_now = StandardScaler().fit_transform(feature_train)


clf_l1_LR= LogisticRegression(
#    tol = 0.0001,C = 0.00005, penalty = 'l1',max_iter = 500,class_weight = 'balanced',verbose = 2
) 
clf_l1_LR.fit(feature_train_now,label_train)
label_val_lr = clf_l1_LR.predict(feature_val)  
print(log_loss(label_val,label_val_lr))


clf_l2_LR= LogisticRegression(
#    tol = 0.0001,C = 0.0001, penalty = 'l2',max_iter = 500,class_weight = 'balanced',verbose = 2
) 
clf_l2_LR.fit(feature_train_now,label_train)
label_val_lr = clf_l2_LR.predict(feature_val)
print(log_loss(label_val,label_val_lr))

1.1711058309
1.1711058309


## 3.训练xgboost
这里要重点讲一下 Xgboost 的调参。通常认为对它性能影响较大的参数有：

- eta：每次迭代完成后更新权重时的步长。越小训练越慢。
- num_round：总共迭代的次数。
- subsample：训练每棵树时用来训练的数据占全部的比例。用于防止 Overfitting。
- colsample_bytree：训练每棵树时用来训练的特征的比例，类似 RandomForestClassifier 的 max_features。
- max_depth：每棵树的最大深度限制。与 Random Forest 不同，Gradient Boosting 如果不对深度加以限制，最终是会 Overfit 的。
- early_stopping_rounds：用于控制在 Out Of Sample 的验证集上连续多少个迭代的分数都没有提高后就提前终止训练。用于防止 Overfitting。

一般的调参步骤是：

- 将训练数据的一部分划出来作为验证集。
- 先将 eta 设得比较高（比如 0.1），num_round 设为 300 ~ 500。
- 用 Grid Search 对其他参数进行搜索
- 逐步将 eta 降低，找到最佳值。
- 以验证集为 watchlist，用找到的最佳参数组合重新在训练集上训练。注意观察算法的输出，看每次迭代后在验证集上分数的变化情况，从而得到最佳的 early_stopping_rounds。

In [77]:
feature_test.columns

Index(['f0', 'f1', 'f2:2_22731265849056483', 'f2:2_509660095530134768',
       'f2:2_1968056100269760729', 'f2:2_2011981573061447208',
       'f2:2_2436715285093487584', 'f2:2_2642175453151805566',
       'f2:2_3203673979138763595', 'f2:2_4879721024980945592',
       'f2:2_5755694407684602296', 'f2:2_5799347067982556520',
       'f2:2_7258015885215914736', 'f2:2_8277336076276184272',
       'f2:2_8710739180200009128', 'f2:3_-1', 'f2:3_6233669177166538628',
       'f2:3_8868887661186419229', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11:woman', 'f11:man', 'f11:family', 'f11:other', 'f12', 'f13:2002.0',
       'f13:2003.0', 'f13:2004.0', 'f13:2005.0', 'f14', 'f15', 'f16',
       'f16:day', 'f16:hour', 'f16:minute', 'f16:second', 'f16:dayofweek',
       'f16:dayofyear', 'f17', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25',
       'f19', 'f1', 'f4'],
      dtype='object')

In [10]:
train = xgb.DMatrix(feature_train,label = label_train)
val = xgb.DMatrix(feature_val,label = label_val)
test = xgb.DMatrix(feature_test)

In [11]:
# coding=utf-8


param = {
    # 1.General Parameters
    'booster': 'gbtree', # 提升计算的模型，可以是：gbtree, gblinear or dart
    'silent': 0,  # 0为打印信息，1为缄默
    # 'nthread': 4, # XGBoost运行时的线程数
    # 'num_pbuffer': # 预测缓冲区的大小，通常设置为训练实例的数量。缓冲区用于保存最后一次提升步骤的预测结果。
    # 'num_feature': # 特征值数量
    
    # 2.Booster Parameters
    'eta': 0.05,  # 学习率，默认0.3，取值范围为：[0,1]，典型值为0.01-0.2，越小越保守
    'gamma': 0.1,  # 节点分裂所需的最小损失函数下降值，和损失函数息息相关。默认0，典型值0.1、0.2，越大越保守，
    'max_depth': 5,  # 树的最大深度，默认6，典型值为3-10，越大越易过拟合。
    'min_child_weight':1, # 这个参数非常影响结果，最小叶子节点样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。在现行回归模型中，这个参数是指建立每个模型所需要的最小样本数。默认1，越大算法越易欠拟合
    'max_delta_step ':0, # 限制每棵树权重改变的最大步长。越大越保守。通常，这个参数不需要设置。但是当各类别的样本十分不平衡时，它对逻辑回归是很有帮助的。
    'subsample':0.8, # 用于训练模型的子样本占整个样本集合的比例，行采样，典型值0.5-1，越小越保守。
    'colsample_bytree ': 0.8,  # 在建立树时对特征采样的比例，列采样。典型值0.5-1

    'lambda':10, # 权重的L2正则化项，默认是1
    'alpha':1,# 权重的L1正则化项，默认是1
    'lambda_bias': 0,  # 在偏置上的L2正则。缺省值为0（在L1上没有偏置项的正则，因为L1时偏置不重要）
    # 'scale_pos_weight':0.5, # 各类样本十分不平衡时，把这个参数设置为一个正数，可以使算法更快收敛。默认是1

    # 3.Task Parameters
    # 'tree_method': 'approx',
    'objective': 'binary:logistic',  # 使用的模型，分类的数目
    'eval_metric':'logloss', # 校验数据所需要的评价指标，不同的目标函数将会有缺省的评价指标
    'base_score':0.5, # 对于所有样本预测为正样本的全局偏置（初始分数）。如果迭代次数够多，改变这个参数对结果不会有影响。
    'seed':0 # 随机数的种子
}
num_boost_round = 500  # 迭代的次数，弱分类器的数量

watchlist = [(train, 'train'),(val, 'eval')]  # 看板，每次迭代都可以在控制台打印出训练集与测试集的损失

In [12]:
# obj、feval、early_stopping_rounds、evals_result、verbose_eval、xgb_model

bst = xgb.train(param, train, num_boost_round, evals=watchlist, obj=None, feval=None, maximize=False,
      early_stopping_rounds=50, evals_result=None, verbose_eval=True, xgb_model=None, callbacks=None,learning_rates=None)
bst.save_model('data/model_xgboost')

[0]	train-logloss:0.648047	eval-logloss:0.647781
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 50 rounds.
[1]	train-logloss:0.607243	eval-logloss:0.606727
[2]	train-logloss:0.570172	eval-logloss:0.56945
[3]	train-logloss:0.536356	eval-logloss:0.535411
[4]	train-logloss:0.505402	eval-logloss:0.504244
[5]	train-logloss:0.476974	eval-logloss:0.475646
[6]	train-logloss:0.450839	eval-logloss:0.449325
[7]	train-logloss:0.426746	eval-logloss:0.425043
[8]	train-logloss:0.404455	eval-logloss:0.402585
[9]	train-logloss:0.383832	eval-logloss:0.381817
[10]	train-logloss:0.364703	eval-logloss:0.362523
[11]	train-logloss:0.346939	eval-logloss:0.344619
[12]	train-logloss:0.330436	eval-logloss:0.327957
[13]	train-logloss:0.315063	eval-logloss:0.312448
[14]	train-logloss:0.300738	eval-logloss:0.297979
[15]	train-logloss:0.287372	eval-logloss:0.284478
[16]	train-logloss:0.2749	eval-logloss:0.27187
[17]	train-logl

[161]	train-logloss:0.08816	eval-logloss:0.081947
[162]	train-logloss:0.088135	eval-logloss:0.08195
[163]	train-logloss:0.088123	eval-logloss:0.081946
[164]	train-logloss:0.088106	eval-logloss:0.081943
[165]	train-logloss:0.088085	eval-logloss:0.081942
[166]	train-logloss:0.08806	eval-logloss:0.081948
[167]	train-logloss:0.08804	eval-logloss:0.081948
[168]	train-logloss:0.088017	eval-logloss:0.08195
[169]	train-logloss:0.088	eval-logloss:0.081947
[170]	train-logloss:0.087985	eval-logloss:0.081942
[171]	train-logloss:0.087971	eval-logloss:0.081946
[172]	train-logloss:0.087956	eval-logloss:0.081944
[173]	train-logloss:0.087935	eval-logloss:0.081954
[174]	train-logloss:0.087912	eval-logloss:0.081959
[175]	train-logloss:0.087894	eval-logloss:0.081963
[176]	train-logloss:0.087878	eval-logloss:0.081965
[177]	train-logloss:0.087863	eval-logloss:0.081959
[178]	train-logloss:0.087848	eval-logloss:0.081967
[179]	train-logloss:0.087836	eval-logloss:0.081961
[180]	train-logloss:0.087809	eval-loglo

In [50]:
importance = bst.get_fscore()

In [51]:
importance = sorted(importance,key=lambda s: s[1])

In [52]:
importance

['f 2 : 2 _ 7 2 5 8 0 1 5 8 8 5 2 1 5 9 1 4 7 3 6',
 'f 2 4',
 'f 2 3',
 'f 6',
 'f 0',
 'f 1 5',
 'f 4',
 'f 1 0',
 'f 2 5',
 'f 1 9',
 'f 2 2',
 'f 2 1',
 'f 1',
 'f 1 6 : h o u r',
 'f 1 6',
 'f 1 6 : m i n u t e',
 'f 1 6 : s e c o n d',
 'f 2 0',
 'f 2 : 2 _ 2 6 4 2 1 7 5 4 5 3 1 5 1 8 0 5 5 6 6',
 'f 1 7',
 'f 1 4',
 'f 7',
 'f 1 2',
 'f 5',
 'f 2 : 2 _ 1 9 6 8 0 5 6 1 0 0 2 6 9 7 6 0 7 2 9',
 'f 9',
 'f 2 : 3 _ 8 8 6 8 8 8 7 6 6 1 1 8 6 4 1 9 2 2 9',
 'f 2 : 2 _ 2 4 3 6 7 1 5 2 8 5 0 9 3 4 8 7 5 8 4',
 'f 8',
 'f 1 1 : m a n',
 'f 2 : 2 _ 3 2 0 3 6 7 3 9 7 9 1 3 8 7 6 3 5 9 5',
 'f 1 1 : f a m i l y',
 'f 2 : 2 _ 5 7 9 9 3 4 7 0 6 7 9 8 2 5 5 6 5 2 0',
 'f 2 : 2 _ 4 8 7 9 7 2 1 0 2 4 9 8 0 9 4 5 5 9 2',
 'f 2 : 2 _ 8 7 1 0 7 3 9 1 8 0 2 0 0 0 0 9 1 2 8',
 'f 2 : 2 _ 5 0 9 6 6 0 0 9 5 5 3 0 1 3 4 7 6 8',
 'f 1 3 : 2 0 0 4 . 0',
 'f 2 : 2 _ 8 2 7 7 3 3 6 0 7 6 2 7 6 1 8 4 2 7 2',
 'f 1 3 : 2 0 0 5 . 0',
 'f 1 3 : 2 0 0 2 . 0',
 'f 2 : 2 _ 2 0 1 1 9 8 1 5 7 3 0 6 1 4 4 7 2 0 8',
 '

In [13]:
# 预测
label_test_xgb = bst.predict(test)

In [14]:
# 4.保存
np.savetxt('data/round1_result_xgb.txt', np.c_[data_index, label_test_xgb], delimiter=',', header='instance_id predicted_score',comments='', fmt='%s %f')

# 4.AdaBoost

In [55]:
clf = AdaBoostClassifier() #迭代100次  
clf.fit(feature_train,label_train)
label_val_lr = clf.predict(feature_val)  
print(log_loss(label_val,label_val_lr))

0.581931454242


# 5.训练Random Forest

In [56]:
clf = RandomForestClassifier()
clf.fit(feature_train , label_train)
label_val_rf = clf.predict(feature_val)    
print(log_loss(label_val,label_val_rf))

0.584341164082


# 6.lightbgm

In [3]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(objective='binary',

                        num_leaves=64,

                        learning_rate=0.01,

                        n_estimators=2000)

gbm.fit(feature_train, label_train,

        eval_set=[(feature_val, label_val)],

        eval_metric='binary_logloss',

        early_stopping_rounds=300)

[1]	valid_0's binary_logloss: 0.0855802
Training until validation scores don't improve for 300 rounds.
[2]	valid_0's binary_logloss: 0.0855103
[3]	valid_0's binary_logloss: 0.08545
[4]	valid_0's binary_logloss: 0.0853836
[5]	valid_0's binary_logloss: 0.0853265
[6]	valid_0's binary_logloss: 0.0852707
[7]	valid_0's binary_logloss: 0.085216
[8]	valid_0's binary_logloss: 0.0851641
[9]	valid_0's binary_logloss: 0.0851081
[10]	valid_0's binary_logloss: 0.085059
[11]	valid_0's binary_logloss: 0.085007
[12]	valid_0's binary_logloss: 0.0849601
[13]	valid_0's binary_logloss: 0.0849131
[14]	valid_0's binary_logloss: 0.0848708
[15]	valid_0's binary_logloss: 0.0848308
[16]	valid_0's binary_logloss: 0.0847922
[17]	valid_0's binary_logloss: 0.084749
[18]	valid_0's binary_logloss: 0.0847106
[19]	valid_0's binary_logloss: 0.0846788
[20]	valid_0's binary_logloss: 0.0846407
[21]	valid_0's binary_logloss: 0.0846072
[22]	valid_0's binary_logloss: 0.0845702
[23]	valid_0's binary_logloss: 0.084538
[24]	valid

[198]	valid_0's binary_logloss: 0.082406
[199]	valid_0's binary_logloss: 0.0824033
[200]	valid_0's binary_logloss: 0.0824013
[201]	valid_0's binary_logloss: 0.0823944
[202]	valid_0's binary_logloss: 0.0823861
[203]	valid_0's binary_logloss: 0.0823823
[204]	valid_0's binary_logloss: 0.0823814
[205]	valid_0's binary_logloss: 0.0823745
[206]	valid_0's binary_logloss: 0.0823688
[207]	valid_0's binary_logloss: 0.0823635
[208]	valid_0's binary_logloss: 0.0823601
[209]	valid_0's binary_logloss: 0.0823566
[210]	valid_0's binary_logloss: 0.0823492
[211]	valid_0's binary_logloss: 0.0823485
[212]	valid_0's binary_logloss: 0.0823467
[213]	valid_0's binary_logloss: 0.0823418
[214]	valid_0's binary_logloss: 0.0823385
[215]	valid_0's binary_logloss: 0.0823361
[216]	valid_0's binary_logloss: 0.0823305
[217]	valid_0's binary_logloss: 0.0823256
[218]	valid_0's binary_logloss: 0.0823223
[219]	valid_0's binary_logloss: 0.0823207
[220]	valid_0's binary_logloss: 0.0823165
[221]	valid_0's binary_logloss: 0.0

[395]	valid_0's binary_logloss: 0.0821422
[396]	valid_0's binary_logloss: 0.0821437
[397]	valid_0's binary_logloss: 0.0821445
[398]	valid_0's binary_logloss: 0.0821424
[399]	valid_0's binary_logloss: 0.0821453
[400]	valid_0's binary_logloss: 0.082147
[401]	valid_0's binary_logloss: 0.0821481
[402]	valid_0's binary_logloss: 0.0821499
[403]	valid_0's binary_logloss: 0.0821517
[404]	valid_0's binary_logloss: 0.0821543
[405]	valid_0's binary_logloss: 0.0821551
[406]	valid_0's binary_logloss: 0.0821559
[407]	valid_0's binary_logloss: 0.0821577
[408]	valid_0's binary_logloss: 0.0821564
[409]	valid_0's binary_logloss: 0.0821578
[410]	valid_0's binary_logloss: 0.0821544
[411]	valid_0's binary_logloss: 0.0821546
[412]	valid_0's binary_logloss: 0.0821521
[413]	valid_0's binary_logloss: 0.0821517
[414]	valid_0's binary_logloss: 0.0821545
[415]	valid_0's binary_logloss: 0.0821547
[416]	valid_0's binary_logloss: 0.0821533
[417]	valid_0's binary_logloss: 0.0821545
[418]	valid_0's binary_logloss: 0.0

[596]	valid_0's binary_logloss: 0.0822839
[597]	valid_0's binary_logloss: 0.0822815
[598]	valid_0's binary_logloss: 0.0822821
[599]	valid_0's binary_logloss: 0.0822822
[600]	valid_0's binary_logloss: 0.0822832
[601]	valid_0's binary_logloss: 0.082285
[602]	valid_0's binary_logloss: 0.0822956
[603]	valid_0's binary_logloss: 0.0822955
[604]	valid_0's binary_logloss: 0.0822956
[605]	valid_0's binary_logloss: 0.0822954
[606]	valid_0's binary_logloss: 0.0822978
[607]	valid_0's binary_logloss: 0.0822992
[608]	valid_0's binary_logloss: 0.0823011
[609]	valid_0's binary_logloss: 0.0823045
[610]	valid_0's binary_logloss: 0.0823052
[611]	valid_0's binary_logloss: 0.0823084
[612]	valid_0's binary_logloss: 0.0823084
[613]	valid_0's binary_logloss: 0.0823046
[614]	valid_0's binary_logloss: 0.0823049
[615]	valid_0's binary_logloss: 0.0823057
[616]	valid_0's binary_logloss: 0.0823076
[617]	valid_0's binary_logloss: 0.0823191
[618]	valid_0's binary_logloss: 0.0823205
[619]	valid_0's binary_logloss: 0.0

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.01,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=64,
       objective='binary', random_state=0, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=50000,
       subsample_freq=1)

In [8]:
label_val_bgm = gbm.predict(feature_val, num_iteration=gbm.best_iteration_)

label_test = gbm.predict(feature_test, num_iteration=gbm.best_iteration_)

print(log_loss(label_val,label_val_bgm))

0.0821334490226


In [9]:
np.savetxt('data/round1_result_lgb.txt', np.c_[data_index, label_test], delimiter=',', header='instance_id predicted_score',comments='', fmt='%s %f')

# 7.GBDT

In [61]:
from sklearn.ensemble import GradientBoostingClassifier    
model = GradientBoostingClassifier()    
model.fit(feature_train, label_train)    
label_val_gbdt = clf.predict(feature_val)  
print(log_loss(label_val,label_val_gbdt))

0.584341164082


# 8.FM/FFM


In [62]:
label_train[label_train < 0.5] = -1.0

In [63]:
feature_train = csc_matrix(feature_train)

In [64]:
feature_val = csc_matrix(feature_val)

In [65]:
from fastFM import sgd,mcmc
fm = sgd.FMClassification(
#     n_iter=1000, init_stdev=0.1, l2_reg_w=0,l2_reg_V=0, rank=2, step_size=0.1
)
fm.fit(feature_train, label_train)
label_val_fm = fm.predict(feature_val)

# fm = mcmc.FMClassification(n_iter=1000, rank=2, init_stdev=0.1)
# y_pred = fm.fit_predict(X_train, y_train, X_test)
# label_val_fm = fm.fit_predict_proba(X_train, y_train, X_test)

/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1735: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
/opt/conda/lib/python3.6/site-packages/fastFM/base.py:123: RuntimeWarning: invalid value encountered in greater
  y_pred[y_proba > .5] = self.classes_[1]


In [66]:
pd.Series(label_val_fm).value_counts()

-1.0    57334
dtype: int64

# 9.集成
要求：
- Base Model 之间的相关性要尽可能的小
- Base Model 之间的性能表现不能差距太大

## 9.1Bagging
- 使用训练数据的不同随机子集来训练每个 Base Model，最后进行每个 Base Model 权重相同的 Vote。

## 9.2Boosting
- 迭代地训练 Base Model，每次根据上一个迭代中预测错误的情况修改训练样本的权重。

## 9.3Blending
- 用不相交的数据训练不同的 Base Model，将它们的输出取（加权）平均。实现简单，但对训练数据利用少了。

## 9.4Stacking
- 我用了两层的模型融合，Level 1使用了：XGBoost、LightGBM、RandomForest、ExtraTrees、DecisionTree、AdaBoost，一共6个模型，Level 2使用了LinearRegression来拟合第一层的结果。
- http://blog.csdn.net/a358463121/article/details/53054686